需要資料
- 公司中文全稱.csv
- 交易日 20050101-20280101.csv

In [ ]:
# token = 'HsKK9aVLMkapUon6YT0J96FrutwumuIVqQksVutM2ft'  # 群組 : 除權息策略通知 # 可轉債詢圈

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime


# 設置 Chrome 選項
chrome_options = Options()
chrome_options.add_argument("--headless")  # 啟用無頭模式

# 建立連結 開啟網頁
driver = webdriver.Chrome(options=chrome_options)
url = "https://web.twsa.org.tw/edoc2/default.aspx"
driver.get(url)

# 找到詢圈按鈕 點擊
element = driver.find_element(By.ID, "ctl00_cphMain_rblReportType_2")
element.click()

# 拿取 網頁html
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
driver.quit()
# -------------------------------------
# 把表格整理成 dataframe
df = pd.DataFrame()

soup_table =soup.select('table.captionStyle')[0].select('tr')
for i in range(1, len(soup_table)):
    comp_whole_name = soup_table[i].find_all('td')[1].text
    broker = soup_table[i].find_all('td')[2].text
    guarantee = soup_table[i].find_all('td')[3].text
    sell_amount = soup_table[i].find_all('td')[4].text
    comp_buy = soup_table[i].find_all('td')[5].text
    start_date = soup_table[i].find_all('td')[6].text.split('~')[0]
    end_date = soup_table[i].find_all('td')[6].text.split('~')[1]
    price = soup_table[i].find_all('td')[7].text
    if '公司債' in guarantee: 
        df = pd.concat([df, pd.DataFrame({
            '公司全名' : [comp_whole_name],
            '主辦承銷商' : [broker],
            '擔保(是/否)' : [guarantee[:3]],
            '承銷股數' : [sell_amount],
            '詢圈銷售股數' : [comp_buy],
            '詢圈開始' : [start_date],
            '詢圈結束' : [end_date],
            '價格' : [price]
        })])

df = df.reset_index(drop=True)

# -------------------------------------
# 計算 詢圈佔承銷比例
df['詢圈銷售股數'] = df['詢圈銷售股數'].apply(lambda x : x.replace(',','')).astype(float)
df['承銷股數'] = df['承銷股數'].apply(lambda x : x.replace(',','')).astype(float)
df['詢圈佔承銷比例'] = round(df['詢圈銷售股數'] / df['承銷股數'] *100, 2)

# -------------------------------------
# 標上 scode sname
df_compname = pd.read_csv('公司中文全稱.csv', encoding='utf-16', sep='\t')
df_compname.rename(columns={'公司中文全稱':'公司全名', '證期會代碼':'scode', '公司中文簡稱':'sname'}, inplace=True)
df_compname = df_compname[['scode', 'sname', '公司全名']]
df_compname['公司全名'] = df_compname['公司全名'].apply(lambda x:x.replace(' ', ''))

df['公司全名'] = df['公司全名'].apply(lambda x:x.split('股份有限公司')[0])
df = df.merge(df_compname, on='公司全名', how='left')

df['主辦承銷商'] = df['主辦承銷商'].apply(lambda x:x.replace('綜合證券股份有限公司', '').replace('證券股份有限公司', ''))
df = df[['scode', 'sname', '主辦承銷商', '詢圈佔承銷比例', '承銷股數', '詢圈銷售股數', '詢圈開始', '詢圈結束', '價格', '擔保(是/否)', '公司全名']]

df.to_excel('詢圈標的.xlsx', index=False)

# -------------------------------------
# 寄送訊息
def lineNotifyMessage(token, msg):
    headers = {
        "Authorization": 'Bearer ' + token, 
        # "Content-Type" : "application/x-www-form-urlencoded"
    }

    data = {'message': msg}
    
    try:
        r = requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data)
        r.raise_for_status()  # 如果發生錯誤，會丟出相應的異常
        return r.status_code
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None
    

token = 'HsKK9aVLMkapUon6YT0J96FrutwumuIVqQksVutM2ft'  # 群組 : 除權息策略通知 # 可轉債詢圈

# today = '2024/05/15'
today = datetime.now().strftime("%Y/%m/%d")

# 計算持有 10 天後的日期
trade_date = pd.read_csv('交易日 20050101-20280101.csv')
trade_date['日期'] = trade_date['日期'].apply(lambda x : x.replace('-','/'))
hold_days = 10
days_cnt = trade_date.loc[trade_date['日期']==today, '累計交易日次'].iloc[0] + (hold_days-1)
sell_day = trade_date.loc[trade_date['累計交易日次']==days_cnt, '日期'].iloc[0]


df_today = df[df['詢圈開始']==today].reset_index(drop=True)
if df_today.empty:
    msg = '今日(%s)沒有開始詢圈標的'%(today)
    print(msg)
    lineNotifyMessage(token, msg)
else:
    for i in range(len(df_today)):
    #     print(df_today.at[i, 'scode'])
    #     print(df_today.at[i, 'sname'])
    #     print(df_today.at[i, '主辦承銷商'])
    #     print(df_today.at[i, '詢圈佔承銷比例'])

        msg = ('\n' + 
               # '今日開始詢圈標的' + '\n' +
               '詢圈標的 : ' + str(df_today.at[i, 'scode']) + ' ' + df_today.at[i, 'sname'] + '\n' +
               '主承銷商 : ' + df_today.at[i, '主辦承銷商'] + '\n' +
               '詢圈比例 : ' + str(df_today.at[i, '詢圈佔承銷比例']) + ' %' + '\n' + 
               '詢圈期間 : ' + df_today.at[i, '詢圈開始'] + ' ~ ' + df_today.at[i, '詢圈結束'] + '\n' + 
               '持有 10 天賣出日期 : ' + sell_day
              )
        print(msg)
        lineNotifyMessage(token, msg)


詢圈標的 : 8467 波力-KY
主承銷商 : 富邦
詢圈比例 : 88.38 %
詢圈期間 : 2024/05/17 ~ 2024/05/21
持有 10 天賣出日期 : 2024/05/30
